In [ ]:
## This code is an example how we do FFT with specific duration
## To streamline the process with multiple files, multiprocessing is leveraged to allocate CPU core to each file
## Coder: Jingyi Yan     Date: 10/10/2025     @ DOES Lab, UTD

In [1]:
# All you need to do is to change the file names in the file2 list 
file2 = ['/home/jingyi/Jy/Condition Monitoring/FM_UTD/FM_UTDallas/New Data/Healthy/Baseline/Healthy_Test2.csv', '/home/jingyi/Jy/Condition Monitoring/FM_UTD/FM_UTDallas/New Data/Healthy/Baseline/Healthy_Test4.csv']

In [2]:
import os
import pandas as pd
import numpy as np
import scipy.fftpack
from multiprocessing import Pool

T = 1 / 10000  # Sample Spacing
duration_list = [0.5, 1, 2] # in the unit of seconds, you may change the index of duration d to select the sampling time
# Please change the columns list if you modify the duration list
columns_list = [ 
    list(np.arange(112, 119, 2)),
    list(np.arange(112,119, 1)),
    list(np.arange(112,119, 0.5))
]
d = 0  # index for duration and deside the colomn heading accordingly

# Function to process a single file
def process_file(file_path):
    try:
        df = pd.read_csv(file_path)
        array = df[['CT 1', 'CT 2', 'CT 3']]
        
        
        duration = duration_list[d]
        columns = columns_list[d]
        N = int(duration * 10000)
        
        freqA = pd.DataFrame(columns=[f"{f} IA" for f in columns])
        freqB = pd.DataFrame(columns=[f"{f} IB" for f in columns])
        freqC = pd.DataFrame(columns=[f"{f} IC" for f in columns])

        i = 0
        for start in range(0, len(array) - N):
            f = scipy.fftpack.fft(array.iloc[start:start+N].values, axis=0)
            f2 = 2.0 / N * np.abs(f[:N // 2, :])
            Freqf = np.linspace(0.0, 1.0 / (2.0 * T), N // 2)
            freqA.loc[i] = f2[Freqf >= 112, 0][Freqf[Freqf >= 112] < 118.5]
            freqB.loc[i] = f2[Freqf >= 112, 1][Freqf[Freqf >= 112] < 118.5]
            freqC.loc[i] = f2[Freqf >= 112, 2][Freqf[Freqf >= 112] < 118.5]
            i += 1

        new_df = pd.concat([
            df.loc[N:, ['VA', 'VB', 'VC', 'CT 1', 'CT 2', 'CT 3']].reset_index(drop=True),
            freqA, freqB, freqC
        ], axis=1)
        
        output_name = file_path[:-4] + f'_FFT{["05","1","2"][d]}I_ABC_112-118.csv'
        new_df.to_csv(output_name, index=False)
        print(f"Processed: {output_name}")
        
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Main execution
if __name__ == "__main__":
    files = file2
    with Pool(processes=os.cpu_count()) as pool:
        pool.map(process_file, files)

Processed: /home/jingyi/Jy/Condition Monitoring/FM_UTD/FM_UTDallas/New Data/Healthy/Baseline/Healthy_Test2_FFT05I_ABC_112-118.csv
Processed: /home/jingyi/Jy/Condition Monitoring/FM_UTD/FM_UTDallas/New Data/Healthy/Baseline/Healthy_Test4_FFT05I_ABC_112-118.csv
